In [3]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

In [4]:
df_m = pd.read_csv("./ml-latest-small/movies.csv")

In [5]:
df_m.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
df_m['genres'] = df_m['genres'].apply(lambda x: x.split("|"))

In [7]:
df_m.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [327]:
df_m.sort_values(by = 'movieId')

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),"[Action, Animation, Comedy, Fantasy]"
9738,193583,No Game No Life: Zero (2017),"[Animation, Comedy, Fantasy]"
9739,193585,Flint (2017),[Drama]
9740,193587,Bungo Stray Dogs: Dead Apple (2018),"[Action, Animation]"


In [489]:
df_m[['movieId', 'title']].to_csv('./content_filtering_data/content_movie_list.csv', header = True, index = False)

In [369]:
vec_df = df_m.explode('genres')

# One-hot encode the genres
vec_df = pd.get_dummies(vec_df, columns=['genres'], dtype = int)
vec_df.drop('genres_(no genres listed)', axis = 1, inplace = True)
vec_df.columns = vec_df.columns.str.replace('genres_', '', regex=False)

In [371]:
vec_df

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story (1995),0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story (1995),0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9738,193583,No Game No Life: Zero (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [329]:
df_m['movieId'].nunique()

9742

In [9]:
df_genres = pd.DataFrame(df_m['genres'].tolist(), index=df_m['movieId']).apply(pd.Series.value_counts , axis=1).fillna(0).astype(int)



In [10]:
df_genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df_genres.drop('(no genres listed)', axis=1, inplace=True)

In [12]:
df_genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [13]:
df_genres.columns

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [14]:

# import datetime
# tmp = pd.read_csv("./ml-latest-small/ratings.csv")
# tmp["year"] = pd.to_datetime(df_i['timestamp'], unit='s')

In [15]:
# tmp[tmp["userId"] == 60]

In [229]:
df_i = pd.read_csv("./ml-latest-small/ratings.csv")

In [231]:
import datetime
df_i["year"] = pd.to_datetime(df_i['timestamp'], unit='s').dt.year

In [233]:
df_i

,userId,movieId,rating,timestamp,year
0,1,1,4.0,964982703,2000
1,1,3,4.0,964981247,2000
2,1,6,4.0,964982224,2000
3,1,47,5.0,964983815,2000
4,1,50,5.0,964982931,2000
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2017
100832,610,168248,5.0,1493850091,2017
100833,610,168250,5.0,1494273047,2017
100834,610,168252,5.0,1493846352,2017


In [235]:
# df_i.sort_values(by = "timestamp",  inplace = True)

In [237]:
# result = df_i.groupby('userId')['year'].agg(['min', 'max']).reset_index()

In [239]:
# result.to_csv("stats.csv", index = False , date_format = '%Y-%m-%d %H:%M:%S')

In [241]:
# result[:50]

In [243]:
df_i[df_i["userId"] == 1]

,userId,movieId,rating,timestamp,year
0,1,1,4.0,964982703,2000
1,1,3,4.0,964981247,2000
2,1,6,4.0,964982224,2000
3,1,47,5.0,964983815,2000
4,1,50,5.0,964982931,2000
...,...,...,...,...,...
227,1,3744,4.0,964980694,2000
228,1,3793,5.0,964981855,2000
229,1,3809,4.0,964981220,2000
230,1,4006,4.0,964982903,2000


In [245]:
# df_i.to_csv("output.csv", index = False)

In [247]:
# df_i1 = df_i.drop('userId', axis=1)

In [249]:
# df_i1.groupby('movieId').filter(lambda x: x['year'].nunique()>1)

In [251]:
df_i.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'year'], dtype='object')

In [253]:
new_df = df_i.groupby(['movieId'])['rating'].mean().reset_index()

In [255]:
new_df.rename(columns={'rating': 'avg_rating'}, inplace=True)

In [257]:
avg_ratings = df_i.groupby(['movieId'])['rating'].mean().reset_index(name='average rating')
df_new = pd.merge(df_i, avg_ratings, on=[ 'movieId'])
df_new = df_new[['movieId', 'average rating', 'timestamp', 'year']]

In [259]:
df_new

,movieId,average rating,timestamp,year
0,1,3.920930,964982703,2000
1,3,3.259615,964981247,2000
2,6,3.946078,964982224,2000
3,47,3.975369,964983815,2000
4,50,4.237745,964982931,2000
...,...,...,...,...
100831,166534,3.333333,1493848402,2017
100832,168248,4.142857,1493850091,2017
100833,168250,3.633333,1494273047,2017
100834,168252,4.280000,1493846352,2017


In [261]:
df_new.columns

Index(['movieId', 'average rating', 'timestamp', 'year'], dtype='object')

In [263]:
df_i

,userId,movieId,rating,timestamp,year
0,1,1,4.0,964982703,2000
1,1,3,4.0,964981247,2000
2,1,6,4.0,964982224,2000
3,1,47,5.0,964983815,2000
4,1,50,5.0,964982931,2000
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2017
100832,610,168248,5.0,1493850091,2017
100833,610,168250,5.0,1494273047,2017
100834,610,168252,5.0,1493846352,2017


In [34]:
# df_i.to_csv("output.csv", index = False)

In [467]:
new_vec_df = pd.merge(df_new, vec_df, on='movieId', how='inner')
new_vec_df.drop('timestamp', axis = 1, inplace = True)
new_vec_df.drop('year', axis = 1, inplace = True)

In [469]:
new_vec_df

,movieId,average rating,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,3.920930,Toy Story (1995),0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3.920930,Toy Story (1995),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3.920930,Toy Story (1995),0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3.920930,Toy Story (1995),0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,3.920930,Toy Story (1995),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274475,168252,4.280000,Logan (2017),0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
274476,170875,2.333333,The Fate of the Furious (2017),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274477,170875,2.333333,The Fate of the Furious (2017),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
274478,170875,2.333333,The Fate of the Furious (2017),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [471]:
new_vec_df['year'] = new_vec_df['title'].str.extract(r'\((\d{4})\)')
# new_vec_df['year'] = new_vec_df['year'].astype(int)

In [473]:
new_vec_df.columns

Index(['movieId', 'average rating', 'title', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'year'],
      dtype='object')

In [475]:
new_vec_df

,movieId,average rating,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,3.920930,Toy Story (1995),0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,1,3.920930,Toy Story (1995),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,1,3.920930,Toy Story (1995),0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
3,1,3.920930,Toy Story (1995),0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,1,3.920930,Toy Story (1995),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274475,168252,4.280000,Logan (2017),0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2017
274476,170875,2.333333,The Fate of the Furious (2017),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017
274477,170875,2.333333,The Fate of the Furious (2017),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2017
274478,170875,2.333333,The Fate of the Furious (2017),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017


In [477]:
# new_vec_df[new_vec_df['year'].isna()]
new_vec_df = new_vec_df.dropna(subset=['year'])
# new_vec_df.drop('title', axis = 1, inplace = True)
new_vec_df = new_vec_df[['movieId', 'year', 'title', 'average rating', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western' ]]

In [479]:
new_vec_df

,movieId,year,title,average rating,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story (1995),3.920930,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1995,Toy Story (1995),3.920930,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1995,Toy Story (1995),3.920930,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1995,Toy Story (1995),3.920930,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1995,Toy Story (1995),3.920930,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274475,168252,2017,Logan (2017),4.280000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
274476,170875,2017,The Fate of the Furious (2017),2.333333,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274477,170875,2017,The Fate of the Furious (2017),2.333333,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
274478,170875,2017,The Fate of the Furious (2017),2.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [481]:
new_vec_df.drop('title', axis = 1, inplace = True)

In [483]:
new_vec_df

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.920930,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1995,3.920930,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1995,3.920930,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1995,3.920930,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1995,3.920930,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274475,168252,2017,4.280000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
274476,170875,2017,2.333333,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274477,170875,2017,2.333333,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
274478,170875,2017,2.333333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [487]:
new_vec_df.to_csv("./content_filtering_data/content_item_vecs.csv", header = False, index = False) 

In [265]:
combined_df = pd.merge(df_new, df_genres, on='movieId', how='inner')

In [267]:
combined_df = combined_df.drop(("timestamp"), axis = 1)
# combined_df = combined_df.drop(("year"), axis = 1)

In [277]:
new_cols = ['movieId', 'year', 'average rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western']
item_df = combined_df[new_cols]

In [297]:
item_df = item_df.rename(columns = {
    'movieId': 'movie id', 
    'avg rating': 'ave rating'
})

In [325]:
item_df.sort_values(by='movie id')

,movie id,year,ave rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2000,3.92093,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
81531,1,2017,3.92093,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
30517,1,2011,3.92093,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
81082,1,2018,3.92093,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
30601,1,1997,3.92093,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27256,193581,2018,4.00000,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27257,193583,2018,3.50000,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27258,193585,2018,3.50000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27259,193587,2018,3.50000,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [301]:
item_df.columns


Index(['movie id', 'year', 'ave rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [303]:
item_df.to_csv("./content_filtering_data/content_item_train.csv", header = False)
filename = './content_filtering_data/content_item_train_header.txt'

# Export the column names as a comma-separated string to a text file
with open(filename, 'w') as f:
    f.write(', '.join(item_df.columns))

In [38]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(combined_df, test_size=0.2, random_state=42, shuffle=True)



In [39]:
train.to_csv("./ml-latest-small/content_item_train.csv", header = False, index = False) 

In [40]:
test.to_csv("./ml-latest-small/content_item_test.csv", header = False, index = False)

In [41]:
df_m.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [377]:
df_m["movieId"]

0            1
1            2
2            3
3            4
4            5
         ...  
9737    193581
9738    193583
9739    193585
9740    193587
9741    193609
Name: movieId, Length: 9742, dtype: int64

In [42]:
df_i.head()

,userId,movieId,rating,timestamp,year
66719,429,595,5.0,828124615,1996-03-29 18:36:55
66716,429,588,5.0,828124615,1996-03-29 18:36:55
66717,429,590,5.0,828124615,1996-03-29 18:36:55
66718,429,592,5.0,828124615,1996-03-29 18:36:55
66712,429,432,3.0,828124615,1996-03-29 18:36:55


In [43]:
df_m.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [44]:
df_i.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'year'], dtype='object')

In [45]:
df_user = pd.DataFrame(df_m['genres'].tolist(), index=df_m['movieId']).apply(pd.Series.value_counts, axis=1).fillna(0).astype(int)


In [46]:
df_user.drop("(no genres listed)", axis = 1, inplace = True) 

In [47]:
df_user.columns

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [48]:
df_user

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [170]:
combined_df_movie = pd.merge( df_i,df_user, on='movieId', how='inner')

In [172]:
combined_df_movie.drop(["timestamp", "year"], axis = 1, inplace = True)

In [174]:
# for genre in combined_df_movie.columns[3:]:  # Skip the first two columns
#     print(genre)
#     combined_df_movie[genre] = combined_df_movie[genre].apply(lambda x: combined_df_movie['rating'] if x == 1 else 0, axis = 1)

# combined_df_movie[combined_df_movie.columns[3:]].multiply(combined_df_movie["rating"], axis="index")


In [177]:
# combined_df_movie[3:] = combined_df_movie[3:] * combined_df_movie["rating"]

In [179]:
combined_df_movie['count'] = combined_df_movie.groupby('userId')['movieId'].transform('count')

In [181]:
combined_df_movie.columns[3:]

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'count'],
      dtype='object')

In [183]:
combined_df_movie

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,count
0,429,595,5.0,0,0,1,1,0,0,0,...,0,1,1,0,1,0,0,0,0,58
1,429,588,5.0,0,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,58
2,429,590,5.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,58
3,429,592,5.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,58
4,429,432,3.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,514,187031,2.5,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,397
100832,514,187595,3.0,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,397
100833,514,5247,2.5,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,397
100834,514,5246,1.5,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,397


In [150]:
# combined_df_movie['Action'] = combined_df_movie.apply(lambda row: row['rating'] if row['Action'] == 1 else 0, axis=1)

In [185]:
for genre in combined_df_movie.columns[3:-1]: 
    combined_df_movie[genre] = combined_df_movie.apply(lambda row: row['rating'] if row[genre] == 1 else 0, axis=1)

In [187]:
combined_df_movie['count'] = combined_df_movie.groupby('userId')['movieId'].transform('count')

In [189]:
combined_df_movie

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,count
0,429,595,5.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,58
1,429,588,5.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,58
2,429,590,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,58
3,429,592,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,58
4,429,432,3.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,514,187031,2.5,2.5,2.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,2.5,0.0,0.0,397
100832,514,187595,3.0,3.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,397
100833,514,5247,2.5,2.5,0.0,0.0,0.0,2.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397
100834,514,5246,1.5,1.5,0.0,0.0,0.0,1.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397


In [162]:
# combined_df_movie['Action'] = combined_df_movie.groupby('userId')['Action'].transform(lambda x: x[x != 0].mean())



In [191]:
for genre in combined_df_movie.columns[3:-1]: 
    # combined_df_movie[genre] = combined_df_movie.apply(lambda row: row['rating'] if row[genre] == 1 else 0, axis=1)
    combined_df_movie[genre] = combined_df_movie.groupby('userId')[genre].transform(lambda x: x[x != 0].mean() if x[x != 0].size > 0 else 0.0)



In [195]:
combined_df_movie["rating"] = combined_df_movie.groupby('userId')["rating"].transform(lambda x: x[x != 0].mean() if x[x != 0].size > 0 else 0.0)


In [207]:
new_cols = ['userId', 'movieId', 'count', 'rating', 'Action', 'Adventure', 'Animation',
            'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
            'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
            'Sci-Fi', 'Thriller', 'War', 'Western']
combined_df_movie = combined_df_movie[new_cols]

In [213]:
combined_df_movie = combined_df_movie.rename(columns={
    'userId': 'user id',
    'count' : 'rating count', 
    'rating': 'rating ave'})

In [217]:
combined_df_movie.drop('movieId', axis = 1, inplace = True)

In [305]:
combined_df_movie.sort_values(by = 'user id', inplace = True)

In [319]:
combined_df_movie

,user id,rating count,rating ave,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
15144,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.0,...,5.00,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
15105,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.0,...,5.00,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
15106,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.0,...,5.00,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
15107,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.0,...,5.00,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
15108,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.0,...,5.00,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89066,610,1302,3.688556,3.600580,3.705993,3.901515,3.651786,3.731144,3.800366,4.2,...,4.35,3.506601,3.628049,3.928571,3.766667,3.731092,3.659363,3.573529,3.776596,3.742424
89067,610,1302,3.688556,3.600580,3.705993,3.901515,3.651786,3.731144,3.800366,4.2,...,4.35,3.506601,3.628049,3.928571,3.766667,3.731092,3.659363,3.573529,3.776596,3.742424
89068,610,1302,3.688556,3.600580,3.705993,3.901515,3.651786,3.731144,3.800366,4.2,...,4.35,3.506601,3.628049,3.928571,3.766667,3.731092,3.659363,3.573529,3.776596,3.742424
89050,610,1302,3.688556,3.600580,3.705993,3.901515,3.651786,3.731144,3.800366,4.2,...,4.35,3.506601,3.628049,3.928571,3.766667,3.731092,3.659363,3.573529,3.776596,3.742424


In [321]:
combined_df_movie.to_csv("./content_filtering_data/content_user_train.csv", header = False)

In [323]:
filename = './content_filtering_data/content_user_train_header.txt'

# Export the column names as a comma-separated string to a text file
with open(filename, 'w') as f:
    f.write(', '.join(combined_df_movie.columns))

In [311]:
df_ratings_sorted = df_i.sort_values(by = 'userId')

In [317]:
df_ratings_sorted['rating'].to_csv('./content_filtering_data/content_y_train.csv', header = False, index = False)




In [507]:
combined_df_movie['user id'].nunique()

610

In [523]:
pickle_df = pd.merge( df_i,df_user, on='movieId', how='inner')

In [525]:
pickle_df

,userId,movieId,rating,timestamp,year,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,2000,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,2000,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,2000,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,2000,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,2000,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2017,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
100832,610,168248,5.0,1493850091,2017,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
100833,610,168250,5.0,1494273047,2017,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
100834,610,168252,5.0,1493846352,2017,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [533]:
user_summary = pickle_df.groupby('userId').agg(
    total_rating=('rating', 'sum'),
    movie_count=('movieId', 'size')
).reset_index()


In [537]:
user_summary['avg rating'] = user_summary['total_rating']/user_summary['movie_count']

In [539]:
user_summary

,userId,total_rating,movie_count,avg rating
0,1,1013.0,232,4.366379
1,2,114.5,29,3.948276
2,3,95.0,39,2.435897
3,4,768.0,216,3.555556
4,5,160.0,44,3.636364
...,...,...,...,...
605,606,4078.0,1115,3.657399
606,607,708.0,187,3.786096
607,608,2604.5,831,3.134176
608,609,121.0,37,3.270270


In [543]:
genre_columns = ['Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [545]:
user_genre_count = pickle_df.groupby('userId')[genre_columns].sum().reset_index()

In [553]:
user_summary['genre_counts'] = user_genre_count[genre_columns].apply(lambda row: row.to_numpy(), axis=1)

In [555]:
user_summary

,userId,total_rating,movie_count,avg rating,genre_counts
0,1,1013.0,232,4.366379,"[90, 85, 29, 42, 83, 45, 0, 68, 47, 1, 17, 0, ..."
1,2,114.5,29,3.948276,"[11, 3, 0, 0, 7, 10, 3, 17, 0, 0, 1, 4, 0, 2, ..."
2,3,95.0,39,2.435897,"[14, 11, 4, 5, 9, 2, 0, 16, 4, 0, 8, 0, 1, 1, ..."
3,4,768.0,216,3.555556,"[25, 29, 6, 10, 104, 27, 2, 120, 19, 4, 4, 1, ..."
4,5,160.0,44,3.636364,"[9, 8, 6, 9, 15, 12, 0, 25, 7, 0, 1, 3, 5, 1, ..."
...,...,...,...,...,...
605,606,4078.0,1115,3.657399,"[151, 147, 42, 49, 421, 133, 5, 698, 97, 8, 52..."
606,607,708.0,187,3.786096,"[72, 45, 6, 19, 55, 27, 0, 82, 21, 0, 35, 1, 5..."
607,608,2604.5,831,3.134176,"[277, 181, 55, 88, 355, 146, 6, 280, 111, 4, 9..."
608,609,121.0,37,3.270270,"[11, 10, 1, 2, 7, 6, 2, 19, 1, 0, 2, 1, 0, 0, ..."


In [557]:
tmp = combined_df_movie.drop_duplicates()

In [565]:
tmp

,user id,rating count,rating ave,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
15144,1,232,4.366379,4.322222,4.388235,4.689655,4.547619,4.277108,4.355556,0.000000,...,5.0000,3.470588,0.000000,4.681818,4.166667,4.307692,4.225000,4.145455,4.500000,4.285714
77220,2,29,3.948276,3.954545,4.166667,0.000000,0.000000,4.000000,3.800000,4.333333,...,0.0000,3.000000,3.750000,0.000000,4.000000,4.500000,3.875000,3.700000,4.500000,3.500000
63946,3,39,2.435897,3.571429,2.727273,0.500000,0.500000,1.000000,0.500000,0.000000,...,0.0000,4.687500,0.000000,0.500000,5.000000,0.500000,4.200000,4.142857,0.500000,0.000000
10415,4,216,3.555556,3.320000,3.655172,4.000000,3.800000,3.509615,3.814815,4.000000,...,4.0000,4.250000,3.000000,4.000000,3.478261,3.379310,2.833333,3.552632,3.571429,3.800000
4974,5,44,3.636364,3.111111,3.250000,4.333333,4.111111,3.466667,3.833333,0.000000,...,0.0000,3.000000,3.666667,4.400000,4.000000,3.090909,2.500000,3.555556,3.333333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49314,606,1115,3.657399,3.178808,3.503401,3.714286,3.448980,3.565321,3.654135,3.800000,...,3.8125,3.346154,3.062500,3.727273,3.791209,3.740845,3.556962,3.525126,3.792308,3.411765
23603,607,187,3.786096,3.722222,3.466667,3.333333,3.421053,3.327273,3.814815,0.000000,...,0.0000,4.114286,5.000000,3.600000,4.647059,3.517241,3.250000,4.114754,4.166667,4.000000
42302,608,831,3.134176,3.330325,3.220994,3.118182,2.460227,2.736620,3.613014,3.000000,...,3.7500,3.319588,4.000000,2.757576,3.550725,2.886792,3.296407,3.536680,3.578947,2.636364
4890,609,37,3.270270,3.090909,3.200000,3.000000,3.000000,3.285714,3.500000,3.000000,...,0.0000,3.500000,3.000000,0.000000,0.000000,3.200000,3.000000,3.285714,3.500000,4.000000


In [561]:
user_summary['genre_avg'] = tmp[genre_columns].apply(lambda row: row.to_numpy(), axis=1)


,userId,total_rating,movie_count,avg rating,genre_counts,genre_avg
0,1,1013.0,232,4.366379,"[90, 85, 29, 42, 83, 45, 0, 68, 47, 1, 17, 0, ...","[3.8823529411764706, 3.9166666666666665, 4.666..."
1,2,114.5,29,3.948276,"[11, 3, 0, 0, 7, 10, 3, 17, 0, 0, 1, 4, 0, 2, ...",NaN
2,3,95.0,39,2.435897,"[14, 11, 4, 5, 9, 2, 0, 16, 4, 0, 8, 0, 1, 1, ...",NaN
3,4,768.0,216,3.555556,"[25, 29, 6, 10, 104, 27, 2, 120, 19, 4, 4, 1, ...",NaN
4,5,160.0,44,3.636364,"[9, 8, 6, 9, 15, 12, 0, 25, 7, 0, 1, 3, 5, 1, ...",NaN
...,...,...,...,...,...,...
605,606,4078.0,1115,3.657399,"[151, 147, 42, 49, 421, 133, 5, 698, 97, 8, 52...",NaN
606,607,708.0,187,3.786096,"[72, 45, 6, 19, 55, 27, 0, 82, 21, 0, 35, 1, 5...","[2.875, 3.272727272727273, 4.333333333333333, ..."
607,608,2604.5,831,3.134176,"[277, 181, 55, 88, 355, 146, 6, 280, 111, 4, 9...",NaN
608,609,121.0,37,3.270270,"[11, 10, 1, 2, 7, 6, 2, 19, 1, 0, 2, 1, 0, 0, ...",NaN


In [501]:
import pickle

with open('content_user_to_genre.pickle', 'rb') as f:
    data = pickle.load(f)

In [503]:
data

defaultdict(None,
            {2: {'glist': array([[3.9  , 5.   , 0.   , 0.   , 4.   , 4.2  , 4.   , 4.   , 0.   ,
                      3.   , 4.   , 0.   , 4.25 , 3.875]]),
              'g_count': array([[5., 1., 0., 0., 5., 5., 2., 9., 0., 1., 1., 0., 2., 8.]]),
              'rating_count': 16,
              'rating_sum': 65.0,
              'movies': {6874: 4.0,
               8798: 3.5,
               46970: 4.0,
               48516: 4.0,
               60756: 5.0,
               71535: 3.0,
               74458: 4.0,
               77455: 3.0,
               80489: 4.5,
               80906: 5.0,
               89774: 5.0,
               91658: 2.5,
               106782: 5.0,
               112552: 4.0,
               115713: 3.5,
               122882: 5.0},
              'rating_ave': 4.0625},
             3: {'glist': array([[0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0.5,
                      0.5]]),
              'g_count': array([[1., 0., 0., 0., 0., 0

In [499]:
len(data)

395